In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [12]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [13]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = optim.SGD(model.parameters(), lr=2e-2)

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step()

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.06713506579399109
epoch:  1, loss: 0.06430155783891678
epoch:  2, loss: 0.06176654249429703
epoch:  3, loss: 0.05949873477220535
epoch:  4, loss: 0.05747276172041893
epoch:  5, loss: 0.05569075420498848
epoch:  6, loss: 0.05414021387696266
epoch:  7, loss: 0.05276952683925629
epoch:  8, loss: 0.05154668167233467
epoch:  9, loss: 0.0504477396607399
epoch:  10, loss: 0.049459800124168396
epoch:  11, loss: 0.04856991767883301
epoch:  12, loss: 0.047768134623765945
epoch:  13, loss: 0.04704546183347702
epoch:  14, loss: 0.046393983066082
epoch:  15, loss: 0.04580668359994888
epoch:  16, loss: 0.045277222990989685
epoch:  17, loss: 0.04479983076453209
epoch:  18, loss: 0.04436938092112541
epoch:  19, loss: 0.04398125037550926
epoch:  20, loss: 0.043631263077259064
epoch:  21, loss: 0.04331570491194725
epoch:  22, loss: 0.04303119704127312
epoch:  23, loss: 0.04277460649609566
epoch:  24, loss: 0.0425431951880455
epoch:  25, loss: 0.04233445227146149
epoch:  26, loss: 0.04

In [14]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = -919.1025453992007
Test metrics:  R2 = -936.8878815968839


In [15]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=1e-2)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step()

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().cpu().numpy().item()))

epoch:  0, loss: 0.2655571699142456
epoch:  1, loss: 0.20283934473991394
epoch:  2, loss: 0.1419653743505478
epoch:  3, loss: 0.08759559690952301
epoch:  4, loss: 0.04829483851790428
epoch:  5, loss: 0.043578069657087326
epoch:  6, loss: 0.07290425151586533
epoch:  7, loss: 0.07765365391969681
epoch:  8, loss: 0.06205832213163376
epoch:  9, loss: 0.04667859524488449
epoch:  10, loss: 0.03957419842481613
epoch:  11, loss: 0.0394400916993618
epoch:  12, loss: 0.04261208325624466
epoch:  13, loss: 0.04610878974199295
epoch:  14, loss: 0.048309653997421265
epoch:  15, loss: 0.048936519771814346
epoch:  16, loss: 0.048118770122528076
epoch:  17, loss: 0.04618070647120476
epoch:  18, loss: 0.04358457401394844
epoch:  19, loss: 0.04085351526737213
epoch:  20, loss: 0.038514621555805206
epoch:  21, loss: 0.03702387586236
epoch:  22, loss: 0.0366700254380703
epoch:  23, loss: 0.03737964481115341
epoch:  24, loss: 0.038638636469841
epoch:  25, loss: 0.039584722369909286
epoch:  26, loss: 0.03949

In [16]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.7060751516593458
Test metrics:  R2 = 0.6821113031185273
